<a href="https://colab.research.google.com/github/sj-minRva/Cancer-Classification/blob/main/Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle

In [8]:
# Load the dataset
df = pd.read_csv("C:/RHEA/S7/Project/Code/Datasets/BRCA_gene_expression.csv", index_col="Unnamed: 0")

print(df.columns)


Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FIRRM', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'C8orf44-SGK3', 'SNORA74C-2', 'ELOA3BP', 'NPBWR1', 'ELOA3DP', 'LNCDAT',
       'LOC124902537', 'RNF228', 'PANO1', 'classes'],
      dtype='object', length=31575)


In [9]:
# Encode the target variable
le = LabelEncoder()
df["classes"] = le.fit_transform(df["classes"])  # Convert categorical labels to 0 and 1


In [10]:
# Define features and target
X = df.drop(columns=["classes"])
y = df["classes"]

In [11]:
# Split dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [12]:
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (856, 31574)
Testing set shape: (368, 31574)


In [13]:
df = df.iloc[:,1:]


In [14]:
df.dropna()

,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,C8orf44-SGK3,SNORA74C-2,ELOA3BP,NPBWR1,ELOA3DP,LNCDAT,LOC124902537,RNF228,PANO1,classes
TCGA-A2-A25D-01A-12R-A16F-07,3.321928,10.802516,10.412570,8.761551,9.859535,11.256209,12.131857,11.154185,11.294621,9.082149,...,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,3.459432,4.459432,0
TCGA-BH-A201-01A-11R-A14M-07,4.954196,11.309476,10.865733,9.799282,9.475733,12.087794,11.087463,11.272047,11.890644,9.355351,...,0.0,1.0,0.0,1.584963,0.0,0.000000,0.0,3.169925,4.857981,0
TCGA-AC-A23C-01A-12R-A169-07,6.832890,12.538189,11.606868,10.055282,8.848623,12.170551,12.151017,11.145932,12.099677,10.318543,...,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,7.066089,5.000000,0
TCGA-AR-A5QP-01A-11R-A28M-07,5.672425,11.074141,10.405141,8.891784,7.894818,11.253257,11.149112,11.365229,10.903129,8.842350,...,1.0,0.0,0.0,1.584963,0.0,0.000000,0.0,2.584963,5.000000,0
TCGA-C8-A12P-01A-11R-A115-07,0.000000,11.626622,10.463524,9.317413,8.873444,12.127027,11.338179,10.645658,10.631177,8.980140,...,0.0,0.0,0.0,3.807355,0.0,3.169925,0.0,1.000000,3.807355,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-LL-A5YP-01A-21R-A28M-07,3.000000,11.302068,9.493855,9.306062,8.942515,11.781770,10.662668,9.592457,11.096715,9.292322,...,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,5.643856,3.807355,0
TCGA-AO-A03L-01A-41R-A056-07,2.000000,11.658211,10.784635,9.813781,8.344296,11.393927,10.882643,10.271463,11.979782,8.622052,...,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.000000,5.781360,0
TCGA-BH-A42T-01A-11R-A24H-07,5.357552,11.174926,10.303781,9.821774,9.541097,10.075479,11.700873,10.409391,11.465056,9.002815,...,1.0,0.0,0.0,3.584963,0.0,0.000000,0.0,0.000000,4.754888,0
TCGA-A2-A04W-01A-31R-A115-07,1.000000,10.903129,9.405141,8.189825,7.813781,10.041659,11.118941,9.459432,10.312883,9.385862,...,0.0,0.0,0.0,0.000000,0.0,2.584963,0.0,3.321928,5.129283,0


In [15]:
df.head()

,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,C8orf44-SGK3,SNORA74C-2,ELOA3BP,NPBWR1,ELOA3DP,LNCDAT,LOC124902537,RNF228,PANO1,classes
TCGA-A2-A25D-01A-12R-A16F-07,3.321928,10.802516,10.412570,8.761551,9.859535,11.256209,12.131857,11.154185,11.294621,9.082149,...,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,3.459432,4.459432,0
TCGA-BH-A201-01A-11R-A14M-07,4.954196,11.309476,10.865733,9.799282,9.475733,12.087794,11.087463,11.272047,11.890644,9.355351,...,0.0,1.0,0.0,1.584963,0.0,0.000000,0.0,3.169925,4.857981,0
TCGA-AC-A23C-01A-12R-A169-07,6.832890,12.538189,11.606868,10.055282,8.848623,12.170551,12.151017,11.145932,12.099677,10.318543,...,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,7.066089,5.000000,0
TCGA-AR-A5QP-01A-11R-A28M-07,5.672425,11.074141,10.405141,8.891784,7.894818,11.253257,11.149112,11.365229,10.903129,8.842350,...,1.0,0.0,0.0,1.584963,0.0,0.000000,0.0,2.584963,5.000000,0
TCGA-C8-A12P-01A-11R-A115-07,0.000000,11.626622,10.463524,9.317413,8.873444,12.127027,11.338179,10.645658,10.631177,8.980140,...,0.0,0.0,0.0,3.807355,0.0,3.169925,0.0,1.000000,3.807355,0


In [16]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 4. Get feature importances

feature_importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for importances
importance_df = pd.DataFrame({
    'Gene': feature_names,
    'Importance': feature_importances
})

# ---------------------------
# 5. Select Top 1000 Genes
# ---------------------------
top_genes_df = importance_df.sort_values(by='Importance', ascending=False).head(1000)

# Optional: Save to CSV
top_genes_df.to_csv('top_1000_genes_breast_cancer.csv', index=False)

# ---------------------------
# DONE!
# ---------------------------
print("Top 1000 genes saved to 'top_1000_genes_breast_cancer.csv'")



c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:35:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Top 1000 genes saved to 'top_1000_genes_breast_cancer.csv'


# None + RF

In [17]:
# Initialize and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")

Accuracy: 0.9755
Precision: 0.9032
AUC: 0.9950
Kappa: 0.8482



#None + XGB

In [18]:

# Initialize and train the XGBoost classifier
xgb_clf = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric="logloss")
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")

Accuracy: 0.9837
Precision: 0.9375
AUC: 0.9948
Kappa: 0.9001


# #None + LR

In [19]:
# Initialize and train the Logistic Regression model
from sklearn.model_selection import cross_val_score
import pandas as pd

lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)


# Make predictions
y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

scores = cross_val_score(lr, X, y, cv=5, scoring='accuracy')


# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

print(np.unique(y_pred, return_counts=True))


# Print evaluation metrics
print("cvs:",scores.mean())
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")

(array([0, 1]), array([333,  35]))
cvs: 0.9950920040147206
Accuracy: 0.9864
Precision: 0.9143
AUC: 0.9975
Kappa: 0.9200


# None + SVM

In [20]:
# Initialize and train the SVM model (with probability=True for AUC calculation)
svm_clf = SVC(kernel='rbf', probability=True, random_state=42)
svm_clf.fit(X_train, y_train)

# Make predictions
y_pred = svm_clf.predict(X_test)
y_pred_proba = svm_clf.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")


Accuracy: 0.9864
Precision: 0.9394
AUC: 0.9974
Kappa: 0.9179


#XGB + SVM

In [21]:
#XGB + SVM
from sklearn.preprocessing import StandardScaler
# Train XGBoost to get feature importances
xgb_selector = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8, random_state=42,
                             use_label_encoder=False, eval_metric="logloss")

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Standardize the features (important for SVM)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_selected)
    X_test_scaled = scaler.transform(X_test_selected)

    # Train SVM classifier
    svm_clf = SVC(kernel="rbf", C=1.0, probability=True, random_state=42)
    svm_clf.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = svm_clf.predict(X_test_scaled)
    y_pred_proba = svm_clf.predict_proba(X_test_scaled)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)


c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:39:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_23932\711342746.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.996037  0.989130   0.941176  0.935188
1       50  0.997094  0.989130   0.941176  0.935188
2      100  0.997094  0.989130   0.941176  0.935188
3      500  0.996037  0.986413   0.939394  0.917901
4     1000  0.995685  0.986413   0.939394  0.917901


# XGB + LR

In [22]:
# Train XGBoost to get feature importances
xgb_selector = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8, random_state=42,
                             use_label_encoder=False, eval_metric="logloss")

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Standardize the features (important for Logistic Regression)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_selected)
    X_test_scaled = scaler.transform(X_test_selected)

    # Train Logistic Regression classifier
    lr_clf = LogisticRegression(solver="liblinear", C=1.0, random_state=42)
    lr_clf.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = lr_clf.predict(X_test_scaled)
    y_pred_proba = lr_clf.predict_proba(X_test_scaled)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)


c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:43:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_23932\756624066.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.997799  0.991848   0.942857  0.952025
1       50  0.997446  0.989130   0.941176  0.935188
2      100  0.997799  0.994565   0.944444  0.968428
3      500  0.998151  0.991848   0.918919  0.953244
4     1000  0.998415  0.991848   0.918919  0.953244


# XGB + RF



In [23]:
#XGB + RF
# Train XGBoost to get feature importances
xgb_selector = XGBClassifier(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    use_label_encoder=False, eval_metric="logloss"
)

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Train Random Forest classifier
    rf_clf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2,
                                    random_state=42, n_jobs=-1)
    rf_clf.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = rf_clf.predict(X_test_selected)
    y_pred_proba = rf_clf.predict_proba(X_test_selected)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)

c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:46:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_23932\3114707820.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.995553  0.983696   0.911765  0.902783
1       50  0.996566  0.986413   0.939394  0.917901
2      100  0.996434  0.986413   0.939394  0.917901
3      500  0.996213  0.983696   0.937500  0.900145
4     1000  0.996302  0.983696   0.937500  0.900145
